In [1]:
import numpy as np

In [2]:
def calc_std_feat(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [3]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [5]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot( X.T, (y_pred - y)))
            
    final_error = calc_logloss(y, y_pred)
    #print(i, W, final_error)
    return W, final_error

In [6]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.int8)

In [7]:
X_std = X.T
X_std[:, 1:] = calc_std_feat(X_std[:, 1:])

# 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [8]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

# 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [9]:
iters = np.logspace(3, 4, 10, dtype=np.int16)
alphas = np.logspace(-4, 0, 10)
alphas = np.unique(np.append(alphas, np.linspace(1, 20, 10)))

In [10]:
best_error = np.inf
best_params = {}

for itr in iters:
    for alpha in alphas:
        W, error = eval_model(X_std, y, iterations=itr, alpha=alpha)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': itr,
                'alpha': alpha
            }
            
    print(f'Ошибка {best_error} при параметрах {best_params}')

Ошибка 0.05628671054404477 при параметрах {'iteration': 1000, 'alpha': 15.777777777777779}
Ошибка 0.03587428612198597 при параметрах {'iteration': 1291, 'alpha': 20.0}
Ошибка 0.0320451806001156 при параметрах {'iteration': 1668, 'alpha': 20.0}
Ошибка 0.028109157303437042 при параметрах {'iteration': 2154, 'alpha': 20.0}
Ошибка 0.024199565926765546 при параметрах {'iteration': 2782, 'alpha': 20.0}
Ошибка 0.020460334006882467 при параметрах {'iteration': 3593, 'alpha': 20.0}
Ошибка 0.017008419065970344 при параметрах {'iteration': 4641, 'alpha': 20.0}
Ошибка 0.013929187809418186 при параметрах {'iteration': 5994, 'alpha': 20.0}
Ошибка 0.011260783226439586 при параметрах {'iteration': 7742, 'alpha': 20.0}
Ошибка 0.009006902394517112 при параметрах {'iteration': 10000, 'alpha': 20.0}


# 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [11]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [12]:
calc_pred_proba(W, X_std)

array([4.31937509e-02, 6.35867212e-13, 1.00000000e+00, 1.97770063e-32,
       9.96908305e-01, 4.08278462e-12, 1.00000000e+00, 7.91426973e-04,
       9.58852154e-01, 1.00000000e+00])

# 4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [13]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [14]:
pred = calc_pred(W, X_std)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

# 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [15]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [16]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0 0 1 0 1 0 1 0 1 1]
Predicted [0 0 1 0 1 0 1 0 1 1]


In [17]:
accuracy(y, pred)

1.0

In [18]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1
    return cm

In [19]:
cm = confusion_matrix(y, pred)
cm

array([[5., 0.],
       [0., 5.]])

In [20]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [21]:
precision(y, pred)

1.0

In [22]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [23]:
recall(y, pred)

1.0

In [24]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [25]:
f_score(y, pred)

1.0

# 6. Могла ли модель переобучиться? Почему?

In [ ]:
Могла и переобучилась. 